# Set Environment

In [1]:
import gym
from gym import spaces
from gym.utils import seeding
import random

# Full deck with distinct face cards
CARDS = [1, 2, 3, 4, 5, 6, 7, 8, 9, '10', 'J', 'Q', 'K'] * 4

def card_value(card):
    return 10 if card in ['10', 'J', 'Q', 'K'] else card

def draw_card(deck):
    return deck.pop()

def draw_hand(deck):
    return [draw_card(deck), draw_card(deck)]

def usable_ace(hand):
    return 1 in hand and sum(card_value(c) for c in hand) + 10 <= 21

def sum_hand(hand):
    total = sum(card_value(c) for c in hand)
    return total + 10 if usable_ace(hand) else total

def is_bust(hand):
    return sum_hand(hand) > 21

def score(hand):
    return 0 if is_bust(hand) else sum_hand(hand)

def is_natural(hand):
    return set(hand) == {1, '10'} or set(hand) == {1, 'J'} or set(hand) == {1, 'Q'} or set(hand) == {1, 'K'}

def can_double_down(hand, actionstaken):
    return len(hand) == 2 and actionstaken == 0

class BlackjackEnv(gym.Env):
    metadata = {"render.modes": ["human"]}

    def __init__(self, numdecks=4, natural=True):
        super().__init__()
        self.action_space = spaces.Discrete(4)  # 0: Stick, 1: Hit, 2: Double Down, 3: Split
        self.observation_space = spaces.Tuple((
            spaces.Tuple((spaces.Discrete(32), spaces.Discrete(32))),  # Player hand (2 cards)
            spaces.Discrete(11),  # Dealer's showing card
            spaces.Discrete(2),   # Usable ace
            spaces.Discrete(2)    # Can double down
        ))

        self.natural = natural
        self.numdecks = numdecks
        self.decks = CARDS * self.numdecks
        random.shuffle(self.decks)
        self.seed()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        random.seed(seed)
        return [seed]

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            self.seed(seed)

        if self._deck_is_out():
            self.decks = CARDS * self.numdecks
            random.shuffle(self.decks)

        self.dealer = draw_hand(self.decks)
        first_hand = draw_hand(self.decks)
        self.hands = [first_hand]
        self.current_hand = 0
        self.actionstaken = 0
        self.hand_results = []
        return self._get_obs()

    def step(self, action):
        assert self.action_space.contains(action), f"Invalid action: {action}"
        if self._deck_is_out():
            self.decks = CARDS * self.numdecks
            random.shuffle(self.decks)

        done = False
        reward = 0
        hand = self.hands[self.current_hand]

        if action == 0:  # Stick
            self._finalize_current_hand()

        elif action == 1:  # Hit
            hand.append(draw_card(self.decks))
            if is_bust(hand):
                self.hand_results.append(-1)
                self._advance_hand()

        elif action == 2:  # Double Down
            if not can_double_down(hand, self.actionstaken):
                raise ValueError("Invalid double down attempt.")
            hand.append(draw_card(self.decks))
            if is_bust(hand):
                self.hand_results.append(-2)
            else:
                self._finalize_current_hand(double=True)

        elif action == 3:  # Split
            if len(hand) != 2 or hand[0] != hand[1]:
                raise ValueError("Invalid split attempt.")
            card = hand[0]
            self.hands[self.current_hand] = [card, draw_card(self.decks)]
            self.hands.insert(self.current_hand + 1, [card, draw_card(self.decks)])

        self.actionstaken += 1

        if self.current_hand >= len(self.hands):
            while sum_hand(self.dealer) < 17:
                self.dealer.append(draw_card(self.decks))

            if len(self.hand_results) < len(self.hands):
                self._finalize_current_hand()

            reward = sum(self.hand_results)
            done = True
        
        return self._get_obs(), reward, done, {}

    def _finalize_current_hand(self, double=False):
        hand = self.hands[self.current_hand]
        player_score = score(hand)
        dealer_score = score(self.dealer)
        result = float(player_score > dealer_score) - float(player_score < dealer_score)
        if is_natural(hand) and result == 1 and self.natural:
            result = 1.5
        self.hand_results.append(result * (2 if double else 1))
        self._advance_hand()

    def _advance_hand(self):
        self.current_hand += 1
        self.actionstaken = 0

    def _get_obs(self):
        if self.current_hand >= len(self.hands):
            return ((0, 0), self.dealer[0], 0, 0)

        hand = self.hands[self.current_hand]
        padded = hand[:2] + [0] * (2 - len(hand))
        return (
            tuple(card_value(c) if c != 0 else 0 for c in padded[:2]),
            card_value(self.dealer[0]),
            int(usable_ace(hand)),
            int(can_double_down(hand, self.actionstaken))
        )

    def _deck_is_out(self):
        return len(self.decks) < self.numdecks * len(CARDS) * 0.1

# Set the Mixed Model

In [3]:
import random
import pandas as pd

# === Load Strategy Tables ===
basic_data = [['H']*10, ['H']*10, ['H']*10, ['H']*10, ['H']*10,
              ['H'] + ['D']*4 + ['H']*5,
              ['D']*8 + ['H']*2, ['D']*10, ['H']*2 + ['S']*3 + ['H']*5,
              ['S']*5 + ['H']*5, ['S']*5 + ['H']*5,
              ['S']*5 + ['H']*5, ['S']*5 + ['H']*5,
              ['S']*10, ['S']*10, ['S']*10, ['S']*10, ['S']*10]

strategy_basic = pd.DataFrame(
    index=[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    columns=[2,3,4,5,6,7,8,9,10,'A'],
    data=basic_data
)

ace_data = [['H']*2 + ['S']*3 + ['H']*5,
            ['H']*3 + ['D']*2 + ['H']*5, ['H']*3 + ['D']*2 + ['H']*5,
            ['H']*2 + ['D']*3 + ['H']*5, ['H']*2 + ['D']*3 + ['H']*5,
            ['H'] + ['D']*4 + ['H']*5, ['S'] + ['D']*4 + ['S']*2 + ['H']*3,
            ['S']*10, ['S']*10, ['S']*10]

strategy_ace = pd.DataFrame(
    index=[12,13,14,15,16,17,18,19,20,21],
    columns=[2,3,4,5,6,7,8,9,10,'A'],
    data=ace_data
)

pair_data = [['P']*5 + ['H']*5, ['P']*5 + ['H']*5, ['H']*3 + ['P']*2 + ['H']*5,
             ['D']*8 + ['H']*2, ['P']*5 + ['H']*5, ['P']*6 + ['H']*4,
             ['P']*10, ['P']*5 + ['S'] + ['P']*2 + ['S']*2, ['S']*10, ['P']*10]

strategy_pair = pd.DataFrame(
    index=[4,6,8,10,12,14,16,18,20,22],
    columns=[2,3,4,5,6,7,8,9,10,'A'],
    data=pair_data
)

# === Strategy-based Action Selection ===
def get_action_from_strategy(player_hand, dealer_card):
    value = card_value(dealer_card)
    dealer_val = 'A' if dealer_card in ['A', 1] else value
    dealer_val = 'A' if dealer_val == 11 else dealer_val

    values = [card_value(c) for c in player_hand]
    total = sum_hand(player_hand)

    # Check for pair
    if len(player_hand) == 2 and values[0] == values[1]:
        pair_total = values[0] * 2
        if pair_total in strategy_pair.index:
            action = strategy_pair.loc[pair_total, dealer_val]
            return convert_action(action)

    # Check for usable ace
    if usable_ace(player_hand) and total in strategy_ace.index:
        action = strategy_ace.loc[total, dealer_val]
        return convert_action(action)

    # Default basic strategy
    if total in strategy_basic.index:
        action = strategy_basic.loc[total, dealer_val]
        return convert_action(action)

    return 1  # Default to Hit if no match

def convert_action(action_str):
    """
    Converts strategy action string to BlackjackEnv-compatible action ID.
    """
    action_map = {
        'S': 0,  # Stick
        'H': 1,  # Hit
        'D': 2,  # Double
        'P': 3   # Split
    }
    return action_map.get(action_str, None)

## Evaluation 1

In [4]:
import pandas as pd
import random

# Prepare result storage
results = []

# === Simulation ===
num_games = 10_000
num_decks = 6

for num_deck in range(1, num_decks + 1):
    env = BlackjackEnv(numdecks=num_deck, natural=False)
    
    wins = 0
    losses = 0
    draws = 0
    total_reward = 0

    for game in range(num_games):
        obs = env.reset()  # Slightly different seed per game
        done = False

        while not done:
            current_hand = env.hands[env.current_hand]
            dealer_card = env.dealer[0]
            
            # Determine valid actions
            valid_actions = [0, 1]  # Stick, Hit always valid
            
            if can_double_down(current_hand, env.actionstaken):
                valid_actions.append(2)  # Double down valid

            # Check if split is valid (same rank and 2 cards)
            if len(current_hand) == 2 and card_value(current_hand[0]) == card_value(current_hand[1]):
                valid_actions.append(3)  # Split valid
            
            # Get recommended action from strategy
            recommended_action = get_action_from_strategy(current_hand, dealer_card)
            
            # Use the recommended action only if it's valid, otherwise fall back to hit
            if recommended_action in valid_actions:
                action = recommended_action
            else:
                # If double down was recommended but not valid, hit instead
                if recommended_action == 2:
                    action = 1  # Hit
                # If split was recommended but not valid, hit instead
                elif recommended_action == 3:
                    action = 1  # Hit
                else:
                    action = 1  # Default to Hit if invalid
            
            obs, reward, done, _ = env.step(action)

            if done:
                total_reward += reward
                if reward > 0:
                    wins += 1
                elif reward < 0:
                    losses += 1
                else:
                    draws += 1

    # Store results
    results.append({
        "Decks": num_deck,
        "Games": num_games,
        "Wins": wins,
        "Draws": draws,
        "Losses": losses,
        "Total Reward": total_reward,
        "Win Rate (%)": round((wins / num_games) * 100, 4),
        "Loss Rate (%)": round((losses / num_games) * 100, 4),
        "Draw Rate (%)": round((draws / num_games) * 100, 4),
        "Average Reward": round(total_reward / num_games, 4)
    })

# Convert to DataFrame
df_mixed_1 = pd.DataFrame(results)
df_mixed_1

,Decks,Games,Wins,Draws,Losses,Total Reward,Win Rate (%),Loss Rate (%),Draw Rate (%),Average Reward
0,1,10000,6191,703,3106,3694.0,61.91,31.06,7.03,0.3694
1,2,10000,6188,698,3114,3760.0,61.88,31.14,6.98,0.3760
2,3,10000,6115,730,3155,3607.0,61.15,31.55,7.30,0.3607
3,4,10000,6125,696,3179,3595.0,61.25,31.79,6.96,0.3595
4,5,10000,6041,734,3225,3428.0,60.41,32.25,7.34,0.3428
5,6,10000,6172,663,3165,3681.0,61.72,31.65,6.63,0.3681


## Evaluation 2

In [5]:
import pandas as pd
import random

# Prepare result storage
results = []

# === Simulation ===
num_games = 10_000
num_decks = 6

for num_deck in range(1, num_decks + 1):
    env = BlackjackEnv(numdecks=num_deck, natural=False)
    
    money = 100
    wins = 0
    losses = 0
    draws = 0
    total_reward = 0

    for game in range(1, num_games+1):
        obs = env.reset()  # Slightly different seed per game
        done = False
        money -= 1

        while not done:
            current_hand = env.hands[env.current_hand]
            dealer_card = env.dealer[0]
            
            # Determine valid actions
            valid_actions = [0, 1]  # Stick, Hit always valid
            
            if can_double_down(current_hand, env.actionstaken):
                valid_actions.append(2)  # Double down valid

            # Check if split is valid (same rank and 2 cards)
            if len(current_hand) == 2 and card_value(current_hand[0]) == card_value(current_hand[1]):
                valid_actions.append(3)  # Split valid
            
            # Get recommended action from strategy
            recommended_action = get_action_from_strategy(current_hand, dealer_card)
            
            # Use the recommended action only if it's valid, otherwise fall back to hit
            if recommended_action in valid_actions:
                action = recommended_action
            else:
                # If double down was recommended but not valid, hit instead
                if recommended_action == 2:
                    action = 1  # Hit
                # If split was recommended but not valid, hit instead
                elif recommended_action == 3:
                    action = 1  # Hit
                else:
                    action = 1  # Default to Hit if invalid
            
            obs, reward, done, _ = env.step(action)
            
            if done:
                total_reward += reward
                if reward > 0:
                    wins += 1
                    money += 2
                elif reward < 0:
                    losses += 1
                else:
                    draws += 1
                    money += 1
        
        if money <= 0:
            break

    # Store results
    results.append({
        "Decks": num_deck,
        "Games": game,
        "Wins": wins,
        "Draws": draws,
        "Losses": losses,
        "Total Reward": total_reward,
        "Win Rate (%)": round((wins / game) * 100, 4),
        "Loss Rate (%)": round((losses / game) * 100, 4),
        "Draw Rate (%)": round((draws / game) * 100, 4),
        "Average Reward": round(total_reward / game, 4),
        "Final Money": money,
    })

# Convert to DataFrame
df_mixed_2 = pd.DataFrame(results)
df_mixed_2

,Decks,Games,Wins,Draws,Losses,Total Reward,Win Rate (%),Loss Rate (%),Draw Rate (%),Average Reward,Final Money
0,1,10000,6165,718,3117,3708.0,61.65,31.17,7.18,0.3708,3148
1,2,10000,6145,691,3164,3647.0,61.45,31.64,6.91,0.3647,3081
2,3,10000,6161,683,3156,3655.0,61.61,31.56,6.83,0.3655,3105
3,4,10000,6179,723,3098,3736.0,61.79,30.98,7.23,0.3736,3181
4,5,10000,6087,781,3132,3579.0,60.87,31.32,7.81,0.3579,3055
5,6,10000,6040,735,3225,3424.0,60.40,32.25,7.35,0.3424,2915


# Set the Simple Model

In [10]:
# Reset Basic
basic_data = [['H']*10, ['H']*10, ['H']*10, ['H']*10, ['H']*10,
              ['H'] + ['D']*4 + ['H']*5,
              ['D']*8 + ['H']*2, ['D']*10, ['H']*2 + ['S']*3 + ['H']*5,
              ['S']*5 + ['H']*5, ['S']*5 + ['H']*5,
              ['S']*5 + ['H']*5, ['S']*5 + ['H']*5,
              ['S']*10,['S']*10, ['S']*10, ['S']*10,['S']*10]
strategy_basic = pd.DataFrame(index=[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
                        columns=[2,3,4,5,6,7,8,9,10,'A'], data=basic_data)

# === Strategy-based Action Selection ===
def get_action_from_strategy(player_hand, dealer_card):
    value = card_value(dealer_card)
    dealer_val = 'A' if dealer_card in ['A', 1] else value
    dealer_val = 'A' if dealer_val == 11 else dealer_val

    values = [card_value(c) for c in player_hand]
    total = sum_hand(player_hand)

    # Default basic strategy
    if total in strategy_basic.index:
        action = strategy_basic.loc[total, dealer_val]
        return convert_action(action)

    return 1  # Default to Hit if no match

def convert_action(action_str):
    """
    Converts strategy action string to BlackjackEnv-compatible action ID.
    """
    action_map = {
        'S': 0,  # Stick
        'H': 1,  # Hit
        'D': 2,  # Double
        'P': 3   # Split
    }
    return action_map.get(action_str, None)

## Evaluation 1

In [11]:
import pandas as pd
import random

# Prepare result storage
results = []

# === Simulation ===
num_games = 10_000
num_decks = 6

for num_deck in range(1, num_decks + 1):
    env = BlackjackEnv(numdecks=num_deck, natural=False)
    
    wins = 0
    losses = 0
    draws = 0
    total_reward = 0

    for game in range(num_games):
        obs = env.reset()  # Slightly different seed per game
        done = False

        while not done:
            current_hand = env.hands[env.current_hand]
            dealer_card = env.dealer[0]
            
            # Determine valid actions
            valid_actions = [0, 1]  # Stick, Hit always valid
            
            if can_double_down(current_hand, env.actionstaken):
                valid_actions.append(2)  # Double down valid

            # Check if split is valid (same rank and 2 cards)
            if len(current_hand) == 2 and card_value(current_hand[0]) == card_value(current_hand[1]):
                valid_actions.append(3)  # Split valid
            
            # Get recommended action from strategy
            recommended_action = get_action_from_strategy(current_hand, dealer_card)
            
            # Use the recommended action only if it's valid, otherwise fall back to hit
            if recommended_action in valid_actions:
                action = recommended_action
            else:
                # If double down was recommended but not valid, hit instead
                if recommended_action == 2:
                    action = 1  # Hit
                # If split was recommended but not valid, hit instead
                elif recommended_action == 3:
                    action = 1  # Hit
                else:
                    action = 1  # Default to Hit if invalid
            
            obs, reward, done, _ = env.step(action)

            if done:
                total_reward += reward
                if reward > 0:
                    wins += 1
                elif reward < 0:
                    losses += 1
                else:
                    draws += 1

    # Store results
    results.append({
        "Decks": num_deck,
        "Games": num_games,
        "Wins": wins,
        "Draws": draws,
        "Losses": losses,
        "Total Reward": total_reward,
        "Win Rate (%)": round((wins / num_games) * 100, 4),
        "Loss Rate (%)": round((losses / num_games) * 100, 4),
        "Draw Rate (%)": round((draws / num_games) * 100, 4),
        "Average Reward": round(total_reward / num_games, 4)
    })

# Convert to DataFrame
df_simple_1 = pd.DataFrame(results)
df_simple_1

,Decks,Games,Wins,Draws,Losses,Total Reward,Win Rate (%),Loss Rate (%),Draw Rate (%),Average Reward
0,1,10000,6139,720,3141,3429.0,61.39,31.41,7.20,0.3429
1,2,10000,6175,744,3081,3564.0,61.75,30.81,7.44,0.3564
2,3,10000,6126,659,3215,3324.0,61.26,32.15,6.59,0.3324
3,4,10000,6179,667,3154,3474.0,61.79,31.54,6.67,0.3474
4,5,10000,6104,744,3152,3356.0,61.04,31.52,7.44,0.3356
5,6,10000,6160,670,3170,3427.0,61.60,31.70,6.70,0.3427


## Evaluation 2

In [12]:
import pandas as pd
import random

# Prepare result storage
results = []

# === Simulation ===
num_games = 10_000
num_decks = 6

for num_deck in range(1, num_decks + 1):
    env = BlackjackEnv(numdecks=num_deck, natural=False)
    
    money = 100
    wins = 0
    losses = 0
    draws = 0
    total_reward = 0

    for game in range(1, num_games+1):
        obs = env.reset()  # Slightly different seed per game
        done = False
        money -= 1

        while not done:
            current_hand = env.hands[env.current_hand]
            dealer_card = env.dealer[0]
            
            # Determine valid actions
            valid_actions = [0, 1]  # Stick, Hit always valid
            
            if can_double_down(current_hand, env.actionstaken):
                valid_actions.append(2)  # Double down valid

            # Check if split is valid (same rank and 2 cards)
            if len(current_hand) == 2 and card_value(current_hand[0]) == card_value(current_hand[1]):
                valid_actions.append(3)  # Split valid
            
            # Get recommended action from strategy
            recommended_action = get_action_from_strategy(current_hand, dealer_card)
            
            # Use the recommended action only if it's valid, otherwise fall back to hit
            if recommended_action in valid_actions:
                action = recommended_action
            else:
                # If double down was recommended but not valid, hit instead
                if recommended_action == 2:
                    action = 1  # Hit
                # If split was recommended but not valid, hit instead
                elif recommended_action == 3:
                    action = 1  # Hit
                else:
                    action = 1  # Default to Hit if invalid
            
            obs, reward, done, _ = env.step(action)
            
            if done:
                total_reward += reward
                if reward > 0:
                    wins += 1
                    money += 2
                elif reward < 0:
                    losses += 1
                else:
                    draws += 1
                    money += 1
        
        if money <= 0:
            break

    # Store results
    results.append({
        "Decks": num_deck,
        "Games": game,
        "Wins": wins,
        "Draws": draws,
        "Losses": losses,
        "Total Reward": total_reward,
        "Win Rate (%)": round((wins / game) * 100, 4),
        "Loss Rate (%)": round((losses / game) * 100, 4),
        "Draw Rate (%)": round((draws / game) * 100, 4),
        "Average Reward": round(total_reward / game, 4),
        "Final Money": money,
    })

# Convert to DataFrame
df_simple_2 = pd.DataFrame(results)
df_simple_2

,Decks,Games,Wins,Draws,Losses,Total Reward,Win Rate (%),Loss Rate (%),Draw Rate (%),Average Reward,Final Money
0,1,10000,6142,620,3238,3336.0,61.42,32.38,6.20,0.3336,3004
1,2,10000,6079,719,3202,3323.0,60.79,32.02,7.19,0.3323,2977
2,3,10000,6049,738,3213,3248.0,60.49,32.13,7.38,0.3248,2936
3,4,10000,6079,692,3229,3281.0,60.79,32.29,6.92,0.3281,2950
4,5,10000,6213,676,3111,3546.0,62.13,31.11,6.76,0.3546,3202
5,6,10000,6080,743,3177,3328.0,60.80,31.77,7.43,0.3328,3003
